In [108]:
# To convert from NFA to DFA using subset construction method
# 1. Epsilon closure of the initial state
# 2. For each state in the epsilon closure, find the transition on each input symbol
# 3. Epsilon closure of the states reached in step 2
# 4. Repeat step 2 and 3 until no new states are reached
# 5. The states reached in step 4 are the states of the DFA
# 6. The transition function of the DFA is the transitions found in step 2 and 3

In [103]:
# def Split_range(range_string):
#     if '-' not in range_string:
#         return [range_string]
#     if range_string[0] == '[' and range_string[-1] == ']':
#         range_string = range_string[1:-1]
#     start, end = range_string.split('-')
#     list_range = []
#     try :
#         start = int(start)
#         end = int(end)
#         for i in range(start, end + 1):
#             list_range.append(str(i))
#         return list_range
#     except ValueError:
#         for i in range(ord(start), ord(end) + 1):
#             list_range.append(chr(i))
#         return list_range

# def split_range_based_on_list(list_of_strings, range_string):
#     if '-' in range_string:
#         range_string = range_string[1:-1]
#     range_list = Split_range(range_string)
#     in_range = []
#     for string in list_of_strings:
#         splited_string = Split_range(string)
#         for s in splited_string:
#             if s in range_list:
#                 in_range += s
#     in_range = list(set(in_range))
#     if not in_range:
#         return [range_string]
#     elif sorted(in_range) == range_list:
#         return []
#     else:
#         ranges = []
#         current_start = range_list[0]
#         for char in sorted(in_range):
#             if char != current_start:
#                 if current_start.isdigit():
#                     if int(char) - int(current_start) == 1:
#                         ranges.append(current_start)
#                     else:
#                         ranges.append(f"{current_start}-{str(int(char) - 1)}")
#                 else:
#                     if ord(char) - ord(current_start) == 1:
#                         ranges.append(current_start)
#                     else:
#                         ranges.append(f"{current_start}-{chr(ord(char) - 1)}")
#             current_start = chr(ord(char) + 1) if current_start.isalpha() else str(int(char) + 1)
#         if current_start != range_list[-1] and sorted(in_range)[-1] != range_list[-1]:
#             if current_start.isdigit():
#                 ranges.append(f"{current_start}-{range_list[-1]}")
#             else:
#                 ranges.append(f"{current_start}-{range_list[-1]}")
#         elif current_start == range_list[-1] and sorted(in_range)[-1] != range_list[-1]:
#             ranges.append(current_start)
#         return ranges

# list_of_strings = ['2-3' , '19']
# range_string = '[1-4]'
# print(split_range_based_on_list(list_of_strings, range_string)) 

['1', '4']


In [1]:
import json
from graphviz import Digraph
import os

In [8]:
def build_nfa_from_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    start_state = []
    start_state.append(data['startingState'])
    accept_states = []
    for state in data:
        if state == 'startingState' :
            continue
        if data[state]['isTerminatingState'] :
            accept_states.append(state)
    Symbols = set()
    nfa = {}
    new_dict = {}
    for state in data:
        if state == 'startingState' :
            continue
        nfa[state] = {}
        new_dict[state] = list()
        for key in data[state] :
            if key == 'isTerminatingState' :
                continue
            nfa[state][key] = set()
            for k in data[state][key]:
                nfa[state][key].add(k)
            new_dict[state].append(key)
            if key != 'ε' :
                Symbols.add(key)
        new_dict[state] = sorted(set(new_dict[state]))
    
    return nfa,list(Symbols),start_state , accept_states , new_dict
f = False
if os.path.isfile("RE_NFA/output10.json"):
    nfa, Symbols ,start_state , accept_states , new_dict = build_nfa_from_json('RE_NFA/output10.json')
else :
    f= True
    print('Validation error')


Validation error


In [164]:
# # To use split range function remove the [] first
# for state in new_dict:
#     sorted_entries = sorted(new_dict[state], key=lambda entry: len(Split_range(entry)))
#     new_dict[state] = sorted_entries
# #print('New Dict : ' , new_dict)

# new_nfa = {}
# for state in new_dict:
#     li = []
#     ld = []
#     new_nfa[state] = {}
#     if len(new_dict[state]) == 0:
#         continue
#     for entry in new_dict[state]:   
#         sr = nfa[state][entry]
#         if new_dict[state].index(entry) == 0:
#             li.append(split_range_based_on_list([],entry))
#             ld.append(nfa[state][entry])
#         else:
#             li.append(split_range_based_on_list(new_dict[state][:new_dict[state].index(entry)],entry))
#             ld.append(nfa[state][entry])
#     for i in range(len(li)):
#         l = str(li[i])[1:-1]
#         l = l[1:-1]
#         if '-' in l:
#             l = "[" + l + "]"
#         if ',' in l:
#             l = "'" + l + "'"
#         new_nfa[state][l] = ld[i]
        
# print('NFA : ' , new_nfa)
# print('NFA : ',nfa)
# nfa = new_nfa
# print(len(nfa))
# print(len(new_nfa))


In [9]:
def epsilon_closure(nfa, states):
    #states = {states} if isinstance(states, str) else set(states)
    closure = set(states)
    stack = list(states)
    while stack:
        state = stack.pop()
        if 'ε' in nfa[state]:
            for s in nfa[state]['ε']:
                if s not in closure:
                    closure.add(s)
                    stack.append(s)
    return closure
alt_nfa = {}
def move(nfa, states, symbol):
    moves = set()
    for state in states:
        if symbol in nfa[state]:
            moves.update(nfa[state][symbol])
            #keys_list = new_dict[state]
            # s_list = split_range_based_on_list((item for item in nfa[state].keys() if item != symbol), symbol)
            # leen = 0
            # for s in s_list:
            #     s = Split_range(s)
            #     if all(item in new_dict[state] for item in s) : 
            #         leen += 1
            # if leen == len(s_list):
            #     moves.update(nfa[state][symbol])
        # if symbol in nfa[state]:
    return moves
def sort_and_tuple(data):
    return tuple(sorted(data)) 
def convert_dfa_from_nfa(nfa , D_states , D_transitions) :
    stack = [D_states]
    while stack:
        D_states = stack.pop()
        for symbol in Symbols:
            if symbol == 'ε':
                continue
            moves = move(nfa, D_states, symbol)
            moves = epsilon_closure(nfa, moves)
            if moves:
                if (frozenset(D_states), symbol) not in D_transitions:
                    D_transitions[(frozenset(D_states), symbol)] = moves
                    stack.append(moves)
    D_transitions = {(sort_and_tuple(key[0]), key[1]): sort_and_tuple(value) for key, value in D_transitions.items()}
        
    return D_transitions
def visualize_dfa(D_transitions):
    dot = Digraph()
    for key in dict_of_states.keys():
        dot.node(str(new_names_states[key]), shape='doublecircle' if dict_of_states[key] else 'circle')
    for (src, symbol), dst in D_transitions.items():
        dot.edge(str(new_names_states[(src)]),str(new_names_states[dst]), label=symbol)
    dot.node('', shape='none')
    dot.edge('' , 'S0', label='Start')

    return dot

if f == True :
    print('Validation error')
else : 
    D_states = epsilon_closure(nfa,start_state) # {0, 1}
    D_transitions = {}
    D_transitions = convert_dfa_from_nfa(nfa , D_states , D_transitions)
    D_states = sorted(set(D_transitions.keys()))
    set_of_states = set()
    dict_of_states = {}
    for D_state in D_states:
        set_of_states.add(D_state[0])
    set_of_states = sorted(set_of_states)
    for state in (set_of_states):
        for accept_state in accept_states:
            if accept_state in state:
                dict_of_states[state] = True
                break
            else:
                dict_of_states[state] = False
    real_states = list() 
    for (src, symbol), dst in D_transitions.items():
        if src not in real_states:
            real_states.append(src)
        if dst not in real_states:
            real_states.append(dst)
    new_names_states = {}
    for i, state in enumerate(real_states):
        new_names_states[state] = f'S{i}'
    for state in real_states : 
        for s in state:
            if s in accept_states:
                dict_of_states[state] = True
                break
            else:
                dict_of_states[state] = False
    dot = visualize_dfa(D_transitions)
    dot.format = 'png'
    dot.render('DFA/Graphs/DFA_1')


Validation error


In [70]:
# print('D_transitions : ' , D_transitions)
# # The keys here are tuple the first is a list and the second is a string of the symbol of move,the same list have many symbols going to differnt destination I want you to get all symbols getting away from this state and if there are ranges for ex if [1-4] , 1 the output will be [2-4] , 1 as this is dfa
# def process_transitions(D_transitions, current_state):
#     result = []
#     dsts = []
#     for (state, symbol), dst in D_transitions.items():
#         if state == current_state:
#             result.append(symbol)
#             dsts.append(dst)
#     return result,dsts
# def split_func(i,sorted_entries):
#     if i == 0:
#         sr = split_range_based_on_list([],sorted_entries[i])
#         sr = str(sr)[1:-1]
#         if '-' in sr:
#             sr = "[" + sr + "]"
#         return(sr)
#     else :     
#         sr = split_range_based_on_list(sorted_entries[0:i], sorted_entries[i])
#         sr = str(sr)[1:-1]
        
#         if '-' in sr:
#             sr = "[" + sr + "]"
#         return(sr)

# new_D_transitions = {}
# for state in real_states:
#     result,dsts = process_transitions(D_transitions, state)
#     sorted_entries = sorted(result, key=lambda entry: len(Split_range(entry)))
#     for i in range(len(sorted_entries)):
#         k = split_func(i,sorted_entries)
#         k = k[1:-1]
        
#         if  '-' in k:
#             k=k[1:-1]
#             k = "[" + k + "]"
#         else:
#             k = k 
#         print(k)
#         new_D_transitions[(state,k)] = D_transitions[state,sorted_entries[i]]
# D_transitions = new_D_transitions


# real_states = list() 
# for (src, symbol), dst in D_transitions.items():

#     if src not in real_states:
#         real_states.append(src)
#     if dst not in real_states:
#         real_states.append(dst)
# print("Real States: ", real_states)
# print(len(real_states))

# new_names_states = {}
# for i, state in enumerate(real_states):
#     new_names_states[state] = f'S{i}'

# print(new_names_states)
# for state in real_states : 
#     for s in state:
#         if s in accept_states:
#             dict_of_states[state] = True
#             break
#         else:
#             dict_of_states[state] = False
# print('Dict of States : ',dict_of_states)

In [67]:
# first_set = set()
# second_set = set()
# for key in dict_of_states.keys():
#     if dict_of_states[key]:
#         second_set.add(key)
#     else:
#         first_set.add(key)
# stack = [first_set, second_set]
# def DFA_Minimization(D_transitions, Symbols, dict_of_states):
#     P = [first_set, second_set]
#     W = [second_set]  
    
#     while W:
#         A = W.pop()
#         for c in Symbols:
#             X = set()
#             for state in dict_of_states.keys():
#                 if (state, c) in D_transitions:
#                     if str(D_transitions[(state, c)]) in A:
#                         X.add(state)
#             for Y in P[:]:
#                 intersection = X & Y
#                 difference = Y - X
#                 if intersection:
#                     P.remove(Y)
#                     P.extend([intersection, difference])
#                     if Y in W:
#                         W.remove(Y)
#                         W.extend([intersection, difference])
#                     else:
#                         if len(intersection) <= len(difference):
#                             W.append(intersection)
#                         else:
#                             W.append(difference)
#     for sets in P[:]:
#         if sets == set():
#             P.remove(sets)  
#     return P

# P = DFA_Minimization(D_transitions, Symbols, dict_of_states)
# li = list(dict_of_states.keys())
# node_mapper = {}
# for i in range(len(P)):
#     node_mapper[f'S{i}'] = (P[i])

# converted_dict = {}
# for key, value_set in node_mapper.items():
#     for value in value_set:
#         converted_dict[value] = key


# minimized_transitions = {}
# for (src, symbol), dst in D_transitions.items():
#     for i in range(len(P)):
#         if src in P[i]:
#             src = f'S{i}'
#             break
#     for i in range(len(P)):
#         if dst in P[i]:
#             dst = f'S{i}'
#             break
#     minimized_transitions[(src, symbol)] = converted_dict[str(dst)]
# print('Minimized Transitions: ',minimized_transitions)
# print("Node mapper : ",node_mapper)
# dict_of_min_states = {}
# for key,sets in node_mapper.items():
#     dict_of_min_states[key] = False
#     for set_ in sets:
#         if dict_of_states[set_]:
#             dict_of_min_states[key] = True
#             break
    
# print((dict_of_min_states))   
# print(P)
# print(len(P))    

In [10]:
def minimize_dfa(D_transitions, dict_of_states):
    partitions = [set(), set()]
    for state, is_accepting in dict_of_states.items():
        partitions[is_accepting].add(state)
    while True:
        new_partitions = []
        for partition in partitions:
            if len(partition) <= 1: 
                new_partitions.append(partition)
                continue
            transition_partitions = {}
            for state in partition:
                transitions = [(symbol, frozenset(D_transitions[state, symbol]) if ((state, symbol) in D_transitions.keys()) else None) for symbol in Symbols] 
                transition_partitions[state] = transitions
            grouped_transitions = {}
            for state, transitions in transition_partitions.items():
                if tuple(transitions) not in grouped_transitions.keys():
                    grouped_transitions[tuple(transitions)] = set()
                grouped_transitions[tuple(transitions)].add(state)
            new_partitions.extend(grouped_transitions.values())
        if new_partitions == partitions: 
            break
        partitions = new_partitions
    state_mapping = {}
    for i, partition in enumerate(partitions):
        for state in partition:
            state_mapping[state] = f'Q{i}'

    return partitions , state_mapping
def visualize_min_dfa(D_transitions , dict_of_min_states):
    dot = Digraph()
    for key in dict_of_min_states.keys():
        dot.node(key, shape='doublecircle' if dict_of_min_states[key] else 'circle')
    for (src, symbol), dst in D_transitions.items():
        dot.edge((src), (dst), label=symbol)
    dot.node('', shape='none')
    dot.edge('' , start_state_minimized,label='start')
    return dot
def generate_json(start_state_minimized, combined_transitions, dict_of_min_states):
    dfa_json = {
        "startingState": start_state_minimized,
    }

    for state, is_terminating in dict_of_min_states.items():
        state_transitions = {}
        for transition, next_state in combined_transitions.items():
            if transition[0] == state:
                state_transitions[transition[1]] = next_state

        dfa_json[state] = {
            "isTerminatingState": is_terminating,
            **state_transitions
        }
    return dfa_json
if f == True :
    print('Validation Error')
else : 
    partitions , state_mapping = minimize_dfa(D_transitions, dict_of_states)
    node_mapper = {}
    for key, value in state_mapping.items():
        if value not in node_mapper:
            node_mapper[value] = set()
        node_mapper[value].add(key)
    dict_of_min_states = {}
    for key,sets in node_mapper.items():
        dict_of_min_states[key] = False
        for set_ in sets:
            if dict_of_states[set_]:
                dict_of_min_states[key] = True
                break
        
    minimized_transitions = {}
    for (src, symbol), dst in D_transitions.items():
        for i in range(len(partitions)):
            if src in partitions[i]:
                src = f'Q{i}'
                break
        for i in range(len(partitions)):
            if dst in partitions[i]:
                dst = f'Q{i}'
                break
        minimized_transitions[(src, symbol)] = dst #state_mapping[str(dst)
    q_0 = epsilon_closure(nfa, start_state) 
    start_index = 1000
    br = False 
    for state in partitions:
        if br:
            break
        for s in list(state):
            if s == real_states[0]:
                start_index = partitions.index(state)
                br = True
                break
    start_state_minimized = f'Q{start_index}'
    combined_transitions = {}
    grouped_transitions = {}

    # Group transitions by (source, destination) tuple
    for transition, destination in minimized_transitions.items():
        if (transition[0], destination) not in grouped_transitions:
            grouped_transitions[(transition[0], destination)] = [transition[1]]
        else:
            grouped_transitions[(transition[0], destination)].append(transition[1])

    # Combine symbols for each (source, destination) group
    for (source, destination), symbols in grouped_transitions.items():
        combined_transitions[(source, ','.join(sorted(set(symbols))))] = destination
        


    dot = visualize_min_dfa(combined_transitions , dict_of_min_states)
    dot.format = 'png'
    dot.render('MIN_DFA/Graphs/DFA_MIN_1')

    min_dfa_json = generate_json(start_state_minimized, combined_transitions, dict_of_min_states)
    dfa_json = generate_json(start_state_minimized, combined_transitions, dict_of_min_states)

    # Write JSON to file
    with open('MIN_DFA/JSON/DFA_MIN_1.json', 'w') as json_file:
        json.dump(dfa_json, json_file, indent=4)

    print("JSON file 'dfa.json' generated successfully.")


Validation Error
